<a href="https://colab.research.google.com/github/JinnnnyHong/Study/blob/main/Model_st.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 딥 러닝 기법 - RNN

보통 RNN 모델에 입력하기 위해 시퀀스 데이터를 고정된 길이의 벡터로 변환하는 과정 필요


1) padding : 시퀀스 데이터를 고정된 길이로 만들기 위해 부족한 부분을 0으로 채움. 하지만 데이터가 길이가 매우 다양한 경우 패딩으로 인해 메모리 사용량이 증가하고, 모델 성능이 저하될 수 있음


2) Truncation : 시퀀스 데이터의 길이를 고정된 길이로 자르는 방법. 이 방법도 패딩과 같이 데이터의 일부가 잘려나가게 되는 문제가 발생할 수 있음


3) embedding : 시퀀스 데이터를 고정된 길이의 벡터로 변환하는 방법 중 하나로, 각 시간 단계마다 특정 차원의 벡터로 표현. 이 방법은 패딩과 Truncation과 달리 정보의 손실이 적고, 모델의 성능을 향상시킬 수 있음

In [ ]:
import torch
import torch.nn as nn
from torchvision.models import resnet101
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

class RRNN(nn.Module):
    def __init__(self, day_length, hidden_size, day_hidden_size):
        super(RRNN, self).__init__()
        self.lstm1 = nn.LSTM(4, day_hidden_size, num_layers=1, bias=True, batch_first=True,
                            dropout=0, bidirectional=True)
        self.lstm2 = nn.LSTM(44+2*day_hidden_size, hidden_size, num_layers=1, bias=True, batch_first=True,
                            dropout=0, bidirectional=True)
        self.fc = nn.Linear(2*hidden_size, 2)
        self.day_length = day_length
        self.day_hidden_size = day_hidden_size
        self.hidden_size = hidden_size
    def init_hidden(self, batch_size, hidden_size):
        return (Variable(torch.randn(2, batch_size, hidden_size)).cuda(),
                Variable(torch.randn(2, batch_size, hidden_size)).cuda())

    def forward(self, x, d_):
        input_storage = []
        for i in range(x.shape[1]):
            self.hidden = self.init_hidden(x.shape[0], self.day_hidden_size)
            x_day = x[:, i, :, :]
            x_day = torch.transpose(x_day, 1,2)
            x_day, h = self.lstm1(x_day, self.hidden)
            x_day = torch.split(x_day, self.day_hidden_size, dim=2)
            x_day = torch.cat([x_day[0][:, -1, :], x_day[1][:, 0, :]], dim=1)
            input_storage.append(x_day)
        f = torch.stack(input_storage, dim=1).squeeze(dim=2)
        mix_f = torch.cat([f, d_], dim=2)
        self.hidden = self.init_hidden(mix_f.shape[0], self.hidden_size)
        xx, h = self.lstm2(mix_f, self.hidden)
        xx = torch.split(xx, self.hidden_size, dim=2)
        xx = torch.cat([xx[0][:, -1, :], xx[1][:, 0, :]], dim=1)
        out = self.fc(xx)

        return out


2. Random Forest

의사결정나무의 단점인 과적합을 보완할 수 있으며, 변수의 중요도를 계산할 수 있음

랜덤 포레스트 모델은 입력 변수의 스케일에 영향을 받지 않기 때문에, 정규화를 하지 않아도 된다. 

하지만, 이전과 마찬가지로 문자열과 같은 비숫자형 데이터를 숫자형 데이터로 변환해야 한다. 이를 위해 One-hot 인코딩, 라벨 인코딩 등의 방법을 사용할 수 있다.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 데이터 로드
iris = load_iris()

# 입력 변수와 타겟 변수 분리
X = iris.data
y = iris.target

# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 모델 생성
rfc = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)

# 모델 학습
rfc.fit(X_train, y_train)

# 검증 데이터에 대한 예측
y_pred = rfc.predict(X_test)

# 모델 성능 평가
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


Accuracy: 100.00%


3. 로지스틱 회귀분석
이진 분류 문제를 해결하기 위한 분석, 데이터는 숫자형이어야 하며, **문자형일시 숫자형 데이터로 변환**하는 과정이 필요함

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 데이터 로드
diabetes = load_diabetes()

# 입력 변수와 타겟 변수 분리
X = diabetes.data
y = diabetes.target

# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 선형 회귀 모델 생성
linreg = LinearRegression()

# 모델 학습
linreg.fit(X_train, y_train)

# 검증 데이터에 대한 예측
y_pred = linreg.predict(X_test)

# 모델 성능 평가
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.2f" % mse)


MSE: 2900.19


랜덤 포레스트 - 시계열 데이터 적용 예시

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# 데이터 불러오기
data = pd.read_csv('data.csv', index_col=0, parse_dates=True)

# 특징(feature)과 타겟(target) 변수 설정
features = data.iloc[:-1]
target = data.iloc[1:]

# 훈련용과 테스트용 데이터 분리
train_size = int(len(features) * 0.7)
train_features = features[:train_size]
train_target = target[:train_size]
test_features = features[train_size:]
test_target = target[train_size:]

# 랜덤 포레스트 모델 생성 및 학습
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(train_features, train_target)

# 예측
predictions = rf.predict(test_features)

# 평가
mse = mean_squared_error(test_target, predictions)
print('Mean Squared Error:', mse)


FileNotFoundError: ignored